# 1: créer une fonction itineraryPrice(station1, station2) qui estimera le prix du trajet en 1ère classe entre les 2 noms de gares passés en paramètres


In [ ]:
pip install pyspark

     |████████████████████████████████| 212.3MB 69kB/s 
     |████████████████████████████████| 204kB 19.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=7adf740234cbf922e01f4375160429ef5e11fc164f2f328e10901a50f9ade7ee
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split



# create SparkSession
spark = SparkSession.builder.master("local[*]") \
                    .appName('SparkSNCF') \
                    .getOrCreate()

# extract SparkContext
sc = spark.sparkContext

print('Spark CPU usage :', sc.defaultParallelism)

Spark CPU usage : 8


In [114]:
import pandas as pd
import re
from pyspark.sql.functions import split


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
PATH_DATA = '/content/drive/MyDrive/data-sets'



##Creation des dataframes


In [ ]:
df = pd.read_scv(PATH_DATA+'/tarifs-tgv-par-od.csv')

In [2]:
#df_spark = spark.read.option("delimiter", ";").option("header",True).csv(PATH_DATA+'/tarifs-tgv-par-od.csv')
df_spark = spark.read.option("delimiter", ";").option("header",True).csv("tarifs-tgv-par-od.csv")
df_spark.show(10)
df_spark.printSchema()



+--------------------+------------------+-----------------------+-----------+------------+
|                  OD|Prix d'appel 2nde |Plein Tarif Loisir 2nde|1ère classe|Commentaires|
+--------------------+------------------+-----------------------+-----------+------------+
|  NIMES-LILLE EUROPE|              30.0|                  144.0|      184.0|        null|
|MOUTIERS SALINS B...|              30.0|                  147.0|      190.0|        null|
|      LANDRY-QUIMPER|              30.0|                  147.0|      190.0|        null|
|TGV HAUTE PICARDI...|              30.0|                  150.0|      188.0|        null|
|TGV HAUTE PICARDI...|              30.0|                  150.0|      188.0|        null|
|PARIS GARE DE LYO...|              25.0|                  121.0|      166.0|        null|
|PARIS GARE DE LYO...|              25.0|                   88.0|      119.0|        null|
|LIMOGES BENEDICTI...|              25.0|                   88.0|      111.0|        null|

## Manipulation d'un DataFrame
* de nombreuses fonctions de transformations sont disponiblke directement sur un data frame 
* https://sparkbyexamples.com/pyspark : la chapitre PySpark DataFrame nous en donne une liste exhaustive

In [3]:
df_split = df_spark.select(split(df_spark.OD,"-").alias("OD"))
df_pandas =  df_split.select("*").toPandas()
df_pandas


# je definie df_pandas pour mieux iterer dessus par la suite


,OD
0,"[NIMES, LILLE EUROPE]"
1,"[MOUTIERS SALINS BRIDES, QUIMPER]"
2,"[LANDRY, QUIMPER]"
3,"[TGV HAUTE PICARDIE, AGDE]"
4,"[TGV HAUTE PICARDIE, SETE]"
...,...
4157,"[PLOUARET TREGOR , PARIS MONTPARNASSE 1 ET 2]"
4158,"[DOL DE BRETAGNE , PARIS MONTPARNASSE 1 ET 2]"
4159,"[MARNE LA VALLEE CHESSY , MORLAIX]"
4160,"[QUIMPERLE , MARNE LA VALLEE CHESSY]"


#Création d'une vue SQL 


In [170]:
# view = spark.registerDataFrameAsTable(df, "table1")
df_spark.createOrReplaceTempView('table')
station = 'COLMAR-MACON VILLE'
string_requete = "SELECT * from table where OD == '{}'".format(station)
requete = spark.sql(string_requete)
prix = str(requete.select("1ère classe").collect())

prix = prix[18:23]
print(prix)



90.0'


# On verifie si les gares et le trajet existent


In [135]:
liste_stations = []
for stations in df_pandas.OD:
  
  for station in stations:
    liste_stations.append(station)
print(liste_stations)


['LILLE FLANDRES', 'AGDE', 'ARRAS', 'SETE', 'PERPIGNAN', 'TGV HAUTE PICARDIE', 'DOLE VILLE', 'PARIS MONTPARNASSE 1 ET 2', 'COLMAR', 'MACON VILLE', 'COLMAR', 'VALENCE TGV', 'BELFORT', 'MONTBELIARD TGV', 'CHALON SUR SAONE', 'DIJON VILLE', 'ANTIBES', 'DIJON VILLE', 'VALENCE TGV', 'BESANCON VIOTTE', 'BELFORT', 'MONTBELIARD TGV', 'NANCY VILLE', 'MONTPELLIER', 'TOUL', 'ANTIBES', 'METZ VILLE', 'NICE VILLE', 'LAVAL', 'ANNEMASSE', 'CASTELNAUDARY', 'AEROPORT CDG 2 TGV ROISSY', 'MARNE LA VALLEE CHESSY', 'PERPIGNAN', 'CANNES', 'AEROPORT CDG 2 TGV ROISSY', 'AEROPORT CDG 2 TGV ROISSY', 'ANTIBES', 'MASSY TGV', 'ST AVRE LA CHAMBRE', 'PARIS GARE DE LYON', 'THONON LES BAINS', 'PARIS GARE DE LYON', 'BELFORT', 'MONTBELIARD TGV', 'PARIS GARE DE LYON', 'VALENCE', 'MARNE LA VALLEE CHESSY', 'VALENCE', 'VERSAILLES CHANTIERS', 'VALENCE', 'LYON ST EXUPERY', 'NANTES', 'LYON PART DIEU', 'CHATELLERAULT', 'AVIGNON', 'LES ARCS DRAGUIGNAN', 'MEUSE', 'EPINAL', 'CHAMBERY CHALLES LES EAUX', 'VENDOME VILLIERS SUR LOIR', '

In [143]:
#  On check si la station existe
def stations_existent(station1, station2): 
  if station1 not in liste_stations:
    print(station1 + "introuvable")
    return False
  elif station2 not in liste_stations:
    print(station2 + "introuvable")
    return False
  else:
    return True

In [171]:
stations_existent('COLMAR', 'MACON VILLE')

True

In [182]:
liste_trajet = []
for stations in df_pandas.OD:
  trajet = stations[0]+'-'+stations[1]
  liste_trajet.append(trajet)

print(liste_trajet)


['LILLE FLANDRES-AGDE', 'ARRAS-SETE', 'PERPIGNAN-TGV HAUTE PICARDIE', 'DOLE VILLE-PARIS MONTPARNASSE 1 ET 2', 'COLMAR-MACON VILLE', 'COLMAR-VALENCE TGV', 'BELFORT-MONTBELIARD TGV', 'DIJON VILLE-ANTIBES', 'DIJON VILLE-VALENCE TGV', 'BESANCON VIOTTE-BELFORT', 'NANCY VILLE-MONTPELLIER', 'TOUL-ANTIBES', 'METZ VILLE-NICE VILLE', 'LAVAL-ANNEMASSE', 'CASTELNAUDARY-AEROPORT CDG 2 TGV ROISSY', 'MARNE LA VALLEE CHESSY-PERPIGNAN', 'CANNES-AEROPORT CDG 2 TGV ROISSY', 'AEROPORT CDG 2 TGV ROISSY-ANTIBES', 'MASSY TGV-ST AVRE LA CHAMBRE', 'PARIS GARE DE LYON-THONON LES BAINS', 'PARIS GARE DE LYON-BELFORT', 'PARIS GARE DE LYON-VALENCE', 'MARNE LA VALLEE CHESSY-VALENCE', 'VERSAILLES CHANTIERS-VALENCE', 'LYON ST EXUPERY-NANTES', 'LYON PART DIEU-CHATELLERAULT', 'AVIGNON-LES ARCS DRAGUIGNAN', 'MEUSE-EPINAL', 'CHAMBERY CHALLES LES EAUX-VENDOME VILLIERS SUR LOIR', 'VENDOME VILLIERS SUR LOIR-MARNE LA VALLEE CHESSY', 'LILLE EUROPE-ECOLE VALENTIN', 'AVIGNON-MENTON', 'LAVAL-ST JEAN DE MAURIENNE ARVAN', 'BESANCON

In [185]:
#  On check si le trajet existe
def trajet_existe(station1, station2):
  if station1+'-'+station2 in liste_trajet:
      return True
      print(i)
  else:
    return False


In [186]:
trajet_existe('COLMAR', 'MACON VILLE')


True


# Fonction ITINARYPRICE() 

In [187]:
def itinaryPrice(station1, station2):
  if stations_existent(station1, station2) and trajet_existe(station1, station2):

    trajet = station1+'-'+station2
    string_requete = "SELECT * from table where OD == '{}'".format(trajet)
    requete = spark.sql(string_requete)
    prix = str(requete.select("1ère classe").collect())

    prix = prix[18:23]
    return prix
  else:
    print("stations introuvables")
    return None



In [188]:
itinaryPrice('ARRAS','SETE')

'188.0'

In [189]:
itinaryPrice('COLMAR', 'MACON VILLE')

"90.0'"

In [190]:
itinaryPrice('MARSEILLE', 'DUNKERQUE')

MARSEILLEintrouvable
stations introuvables


In [192]:
itinaryPrice('LILLE EUROPE','ST JEAN DE LUZ CIBOURE')

'174.0'